In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('Nifty.csv')
df.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
3989,2023-11-10,19351.849609,19451.300781,19329.449219,19425.349609,19425.349609,152200.0
3990,2023-11-13,19486.750000,19494.400391,19414.750000,19443.550781,19443.550781,189300.0
3991,2023-11-15,19651.400391,19693.199219,19579.650391,19675.449219,19675.449219,291500.0
3992,2023-11-16,19674.699219,19875.250000,19627.000000,19765.199219,19765.199219,282700.0
3993,2023-11-17,19674.750000,19806.000000,19667.449219,19731.800781,19731.800781,236800.0


In [3]:
df['Color'] = (df['Open'] > df['Adj Close'].shift(1)).astype(int)

In [4]:
df.tail()

,Date,Open,High,Low,Close,Adj Close,Volume,Color
3989,2023-11-10,19351.849609,19451.300781,19329.449219,19425.349609,19425.349609,152200.0,0
3990,2023-11-13,19486.750000,19494.400391,19414.750000,19443.550781,19443.550781,189300.0,1
3991,2023-11-15,19651.400391,19693.199219,19579.650391,19675.449219,19675.449219,291500.0,1
3992,2023-11-16,19674.699219,19875.250000,19627.000000,19765.199219,19765.199219,282700.0,0
3993,2023-11-17,19674.750000,19806.000000,19667.449219,19731.800781,19731.800781,236800.0,0


In [40]:
# Calculate the difference between previous close and the next day's open
df['Magnitude'] = abs(df['Close'].shift(1) - df['Open'])

# Normalize the 'Magnitude' values to a range from 1 to 10
min_magnitude = df['Magnitude'].min()
max_magnitude = df['Magnitude'].max()

df['Magnitude'] = 1 + 9 * ((df['Magnitude'] - min_magnitude) / (max_magnitude - min_magnitude))

# Ensure that values are capped at 10
df['Magnitude'] = df['Magnitude'].clip(1, 10)


In [41]:
df.tail()

,Open,High,Low,Close,Adj Close,Volume,Color,Magnitude
Date,,,,,,,,
2023-11-10,19351.849609,19451.300781,19329.449219,19425.349609,19425.349609,152200.0,0,1.488978
2023-11-13,19486.750000,19494.400391,19414.750000,19443.550781,19443.550781,189300.0,1,1.690970
2023-11-15,19651.400391,19693.199219,19579.650391,19675.449219,19675.449219,291500.0,1,3.339039
2023-11-16,19674.699219,19875.250000,19627.000000,19765.199219,19765.199219,282700.0,0,1.008440
2023-11-17,19674.750000,19806.000000,19667.449219,19731.800781,19731.800781,236800.0,0,2.017872


In [7]:
!python -m pip install lightgbm

You should consider upgrading via the 'c:\Users\anirudh\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [9]:
df = df.dropna()

In [10]:
df.set_index('Date', inplace=True)

In [11]:
# Features
features = ['Adj Close', 'High', 'Open', 'Low', 'Volume']

# Targets
targets = ['Open', 'Magnitude']

# Separate features and target variables
X = df[features]
y = df[targets]

# Split the dataset into training and testing sets (80:20 ratio)
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X, y, test_size=0.2, shuffle=False)

In [12]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR, NuSVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor

param_grids = {
    'Random Forest Regressor': {'estimator__n_estimators': [10, 50, 100]},
    'Gradient Boosting Regressor': {'estimator__n_estimators': [10, 50, 100], 'estimator__learning_rate': [0.01, 0.1, 0.2]},
    'Linear Regression': {},
    'Ridge Regression': {'estimator__alpha': [0.01, 0.1, 1.0]},
    'Lasso Regression': {'estimator__alpha': [0.01, 0.1, 1.0]},
    # 'Support Vector Regressor': {'estimator__C': [0.1, 1, 10], 'estimator__kernel': ['linear', 'rbf']},
    # 'Nu Support Vector Regressor': {'estimator__C': [0.1, 1, 10], 'estimator__kernel': ['linear', 'rbf']},
    'K-Nearest Neighbors Regressor': {'estimator__n_neighbors': [3, 5, 7]},
    'Decision Tree Regressor': {'estimator__max_depth': [None, 10, 20, 30]},
    'Multi-layer Perceptron Regressor': {'estimator__hidden_layer_sizes': [(50,), (100,), (50, 50)]},
}

multi_output_models = {}
for model_name, model in {
    'Random Forest Regressor': RandomForestRegressor(),
    'Gradient Boosting Regressor': GradientBoostingRegressor(),
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    # 'Support Vector Regressor': SVR(),
    # 'Nu Support Vector Regressor': NuSVR(),
    'K-Nearest Neighbors Regressor': KNeighborsRegressor(),
    'Decision Tree Regressor': DecisionTreeRegressor(),
    'Multi-layer Perceptron Regressor': MLPRegressor(),
}.items():
    # Use MultiOutputRegressor for compatibility with grid search
    multi_output_model = MultiOutputRegressor(model)
    
    # Perform grid search for hyperparameter tuning
    grid_search = GridSearchCV(multi_output_model, param_grids[model_name], cv=3, scoring='r2')
    grid_search.fit(X_train_reg, y_train_reg)
    
    # Save the best model with tuned hyperparameters
    multi_output_models[model_name] = grid_search.best_estimator_

    # Print the best hyperparameters for each model
    print(f"Best hyperparameters for {model_name}: {grid_search.best_params_}")



Best hyperparameters for Random Forest Regressor: {'estimator__n_estimators': 10}
Best hyperparameters for Gradient Boosting Regressor: {'estimator__learning_rate': 0.1, 'estimator__n_estimators': 50}
Best hyperparameters for Linear Regression: {}
Best hyperparameters for Ridge Regression: {'estimator__alpha': 1.0}


c:\Users\anirudh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.715e+06, tolerance: 9.317e+05
  model = cd_fast.enet_coordinate_descent(
c:\Users\anirudh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.154e+01, tolerance: 1.132e-02
  model = cd_fast.enet_coordinate_descent(
c:\Users\anirudh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the 

Best hyperparameters for Lasso Regression: {'estimator__alpha': 1.0}
Best hyperparameters for K-Nearest Neighbors Regressor: {'estimator__n_neighbors': 7}
Best hyperparameters for Decision Tree Regressor: {'estimator__max_depth': 10}


c:\Users\anirudh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\anirudh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\anirudh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\anirudh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the 

Best hyperparameters for Multi-layer Perceptron Regressor: {'estimator__hidden_layer_sizes': (50, 50)}


In [31]:
# Separate features and target variable
X_mag = df.drop('Magnitude', axis=1)
y_mag = df['Magnitude']

param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False],
    # Add more parameters as needed
}

# Split the data into training and testing sets
X_train_mag, X_test_mag, y_train_mag, y_test_mag = train_test_split(X_mag, y_mag, test_size=0.2, random_state=42)

# Initialize and train the model (using RandomForestRegressor as an example)
magnitude_model = RandomForestRegressor()
magnitude_model.fit(X_train_mag, y_train_mag)

# Initialize GridSearchCV
grid_search = GridSearchCV(magnitude_model, param_grid, cv=3, scoring='r2')

# Perform the grid search on the data
grid_search.fit(X_train_mag, y_train_mag)

# Get the best parameters
best_params = grid_search.best_params_

# Train the model with the best parameters
best_magnitude_model = RandomForestRegressor(**best_params)
best_magnitude_model.fit(X_train_mag, y_train_mag)


KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import mean_squared_error

# Make predictions on the test set
y_pred_best_mag = best_magnitude_model.predict(X_test_mag)

# Evaluate the model
best_magnitude_accuracy = best_magnitude_model.score(X_test_mag, y_test_mag)
best_mse = mean_squared_error(y_test_mag, y_pred_best_mag)

print(f'Best Magnitude Model Parameters: {best_params}')
print(f'Best Magnitude Model Accuracy: {best_magnitude_accuracy:.4f}')
print(f'Best Mean Squared Error: {best_mse:.4f}')

Best Magnitude Model Parameters: {'max_depth': 10, 'n_estimators': 50}
Best Magnitude Model Accuracy: 0.3501
Best Mean Squared Error: 0.0450


In [34]:
from sklearn.ensemble import GradientBoostingRegressor

# Expand the parameter grid
param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'alpha': [0.9, 0.95, 0.99],
    # Add more parameters as needed
}

# Initialize the GradientBoostingRegressor
magnitude_model = GradientBoostingRegressor()

# Initialize GridSearchCV
grid_search = GridSearchCV(magnitude_model, param_grid, cv=3, scoring='r2')

# Perform the grid search on the data
grid_search.fit(X_train_mag, y_train_mag)

# Get the best parameters
best_params = grid_search.best_params_

# Train the model with the best parameters
best_magnitude_model = GradientBoostingRegressor(**best_params)
best_magnitude_model.fit(X_train_mag, y_train_mag)


KeyboardInterrupt: 

In [33]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

# Initialize the Ridge Regression model
ridge_model = Ridge()

# Define the hyperparameter grid for Ridge Regression
param_grid = {'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]}

# Perform grid search for hyperparameter tuning
grid_search_ridge = GridSearchCV(ridge_model, param_grid, cv=3, scoring='r2')
grid_search_ridge.fit(X_train_mag, y_train_mag)

# Get the best Ridge Regression model with tuned hyperparameters
best_ridge_model = grid_search_ridge.best_estimator_

# Make predictions on the test set
y_pred_ridge = best_ridge_model.predict(X_test_mag)

# Evaluate the model
ridge_accuracy = best_ridge_model.score(X_test_mag, y_test_mag)
ridge_mse = mean_squared_error(y_test_mag, y_pred_ridge)

print(f'Best Ridge Regression Model Parameters: {grid_search_ridge.best_params_}')
print(f'Ridge Regression Model Accuracy: {ridge_accuracy:.4f}')
print(f'Mean Squared Error: {ridge_mse:.4f}')


c:\Users\anirudh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=3.16348e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
c:\Users\anirudh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=3.31068e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
c:\Users\anirudh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=3.37699e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


Best Ridge Regression Model Parameters: {'alpha': 10.0}
Ridge Regression Model Accuracy: 0.3027
Mean Squared Error: 0.0483


In [39]:
df.to_csv('updatedNifty.csv')


In [ ]:
# Make predictions on the test set
y_pred_best_mag = best_magnitude_model.predict(X_test_mag)

# Evaluate the model
best_magnitude_accuracy = best_magnitude_model.score(X_test_mag, y_test_mag)
best_mse = mean_squared_error(y_test_mag, y_pred_best_mag)

print(f'Best Magnitude Model Parameters: {best_params}')
print(f'Best Magnitude Model Accuracy: {best_magnitude_accuracy:.4f}')
print(f'Best Mean Squared Error: {best_mse:.4f}')

In [16]:
for model_name, model in multi_output_models.items():
    accuracy = model.score(X_test_reg, y_test_reg)*100
    print(f'{model_name} Accuracy: {accuracy}')

Random Forest Regressor Accuracy: -258.2479811435713
Gradient Boosting Regressor Accuracy: -273.09961718271506
Linear Regression Accuracy: 45.230941830596095
Ridge Regression Accuracy: 45.23093863750508
Lasso Regression Accuracy: 26.637604546807975
K-Nearest Neighbors Regressor Accuracy: -380.9403465870781
Decision Tree Regressor Accuracy: -249.6773318627603
Multi-layer Perceptron Regressor Accuracy: -259337.39635142565


In [18]:
import joblib

# Dictionary to store model accuracies
model_accuracies = {}

for model_name, model in multi_output_models.items():
    # Evaluate the model
    accuracy = model.score(X_test_reg, y_test_reg) * 100
    model_accuracies[model_name] = accuracy
    
    # Save the model to a file
    filename = f"regmodel/{model_name}_{accuracy:.4f}.h5"
    joblib.dump(model, filename)

# Print model accuracies
for model_name, accuracy in model_accuracies.items():
    print(f'{model_name} Accuracy: {accuracy:.4f}')

Random Forest Regressor Accuracy: -258.2480
Gradient Boosting Regressor Accuracy: -273.0996
Linear Regression Accuracy: 45.2309
Ridge Regression Accuracy: 45.2309
Lasso Regression Accuracy: 26.6376
K-Nearest Neighbors Regressor Accuracy: -380.9403
Decision Tree Regressor Accuracy: -249.6773
Multi-layer Perceptron Regressor Accuracy: -259337.3964


In [38]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV

# Assuming your dataframe is named 'df'
# Selecting features and target variable
features = df[['Open', 'Close', 'High', 'Low', 'Adj Close', 'Volume']]
target = df['Color']

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)


# Define the parameter grid to search
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize the Random Forest Classifier
rf_classifier = RandomForestClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(rf_classifier, param_grid, cv=3, scoring='accuracy')

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_

# Train the model with the best parameters
best_rf_classifier = RandomForestClassifier(**best_params, random_state=42)
best_rf_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_rf_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_report_output = classification_report(y_test, y_pred)

print(f'Best Parameters: {best_params}')
print(f'Accuracy with Best Parameters: {accuracy:.4f}')
print('Classification Report:\n', classification_report_output)


Best Parameters: {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 50}
Accuracy with Best Parameters: 0.6112
Classification Report:
               precision    recall  f1-score   support

           0       0.42      0.07      0.13       298
           1       0.62      0.94      0.75       489

    accuracy                           0.61       787
   macro avg       0.52      0.51      0.44       787
weighted avg       0.55      0.61      0.51       787



In [11]:
df.set_index('Date', inplace=True)

# Extracting features and target variables
features = df.drop(['Color', 'Magnitude'], axis=1)
target_color = df['Color']
target_safety = df['Manitude']

In [12]:
features.fillna(features.mean(), inplace=True)
target_color.fillna(target_color.mean(), inplace=True)
target_safety.fillna(target_safety.mean(), inplace=True)


In [13]:
X_train_color, X_test_color, y_train_color, y_test_color = train_test_split(features, target_color, test_size=0.2, random_state=42)


In [14]:
X_train_safety, X_test_safety, y_train_safety, y_test_safety = train_test_split(features, target_safety, test_size=0.2, random_state=42)


In [15]:
scaler = StandardScaler()
X_train_scaled_color = scaler.fit_transform(X_train_color)
X_test_scaled_color = scaler.transform(X_test_color)

X_train_scaled_safety = scaler.fit_transform(X_train_safety)
X_test_scaled_safety = scaler.transform(X_test_safety)

In [16]:
classifiers = {
    'Random Forest': RandomForestClassifier(),
    'Decision Tree': DecisionTreeClassifier(),
    'Support Vector Machine': SVC(),
    'Logistic Regression': LogisticRegression(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Naive Bayes': GaussianNB(),
    'AdaBoost': AdaBoostClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'Neural Network': MLPClassifier(),
    'XGBoost': XGBClassifier(),
    'LightGBM': LGBMClassifier()
    # Add more classifiers as needed
}

In [18]:
results_color = {}
for clf_name, clf in classifiers.items():
    grid_search_color = GridSearchCV(clf, param_grid={}, cv=5, n_jobs=-1)
    grid_search_color.fit(X_train_scaled_color, y_train_color)

    # Predictions on the test set for 'Color'
    y_pred_color = grid_search_color.predict(X_test_scaled_color)

    # Accuracy and classification report for 'Color'
    accuracy_color = accuracy_score(y_test_color, y_pred_color)
    report_color = classification_report(y_test_color, y_pred_color)

    # Store the results for 'Color'
    results_color[clf_name] = {'Best Model': grid_search_color.best_estimator_, 'Accuracy': accuracy_color, 'Report': report_color}


[LightGBM] [Info] Number of positive: 1441, number of negative: 692
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000447 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1785
[LightGBM] [Info] Number of data points in the train set: 2133, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.675574 -> initscore=0.733507
[LightGBM] [Info] Start training from score 0.733507


In [20]:
results_color

{'Random Forest': {'Best Model': RandomForestClassifier(),
  'Accuracy': 0.6666666666666666,
  'Report': '              precision    recall  f1-score   support\n\n           0       0.43      0.27      0.34       164\n           1       0.72      0.84      0.78       370\n\n    accuracy                           0.67       534\n   macro avg       0.58      0.56      0.56       534\nweighted avg       0.63      0.67      0.64       534\n'},
 'Decision Tree': {'Best Model': DecisionTreeClassifier(),
  'Accuracy': 0.6329588014981273,
  'Report': '              precision    recall  f1-score   support\n\n           0       0.40      0.39      0.40       164\n           1       0.73      0.74      0.74       370\n\n    accuracy                           0.63       534\n   macro avg       0.57      0.57      0.57       534\nweighted avg       0.63      0.63      0.63       534\n'},
 'Support Vector Machine': {'Best Model': SVC(),
  'Accuracy': 0.6985018726591761,
  'Report': '              pr

In [19]:
results_safety = {}
for clf_name, clf in classifiers.items():
    grid_search_safety = GridSearchCV(clf, param_grid={}, cv=5, n_jobs=-1)
    grid_search_safety.fit(X_train_scaled_safety, y_train_safety)

    # Predictions on the test set for 'Safety'
    y_pred_safety = grid_search_safety.predict(X_test_scaled_safety)

    # Accuracy and classification report for 'Safety'
    accuracy_safety = accuracy_score(y_test_safety, y_pred_safety)
    report_safety = classification_report(y_test_safety, y_pred_safety)

    # Store the results for 'Safety'
    results_safety[clf_name] = {'Best Model': grid_search_safety.best_estimator_, 'Accuracy': accuracy_safety, 'Report': report_safety}

c:\Users\anirudh\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


[LightGBM] [Info] Number of positive: 495, number of negative: 1638
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000510 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1785
[LightGBM] [Info] Number of data points in the train set: 2133, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.232068 -> initscore=-1.196674
[LightGBM] [Info] Start training from score -1.196674
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, be

In [21]:
results_safety

{'Random Forest': {'Best Model': RandomForestClassifier(),
  'Accuracy': 1.0,
  'Report': '              precision    recall  f1-score   support\n\n           0       1.00      1.00      1.00       412\n           1       1.00      1.00      1.00       122\n\n    accuracy                           1.00       534\n   macro avg       1.00      1.00      1.00       534\nweighted avg       1.00      1.00      1.00       534\n'},
 'Decision Tree': {'Best Model': DecisionTreeClassifier(),
  'Accuracy': 1.0,
  'Report': '              precision    recall  f1-score   support\n\n           0       1.00      1.00      1.00       412\n           1       1.00      1.00      1.00       122\n\n    accuracy                           1.00       534\n   macro avg       1.00      1.00      1.00       534\nweighted avg       1.00      1.00      1.00       534\n'},
 'Support Vector Machine': {'Best Model': SVC(),
  'Accuracy': 0.9943820224719101,
  'Report': '              precision    recall  f1-score   

In [23]:
import joblib

# Function to save the best model as h5 file
def save_best_model(model, model_name, accuracy, directory):
    file_name = f"{model_name}_{accuracy:.4f}.h5"
    file_path = f"{directory}/{file_name}"
    joblib.dump(model, file_path)
    print(f"Saved {model_name} model with accuracy {accuracy:.4f} at: {file_path}")

# Save the best models for 'Color'
for clf_name, result in results_color.items():
    save_best_model(result['Best Model'], clf_name, result['Accuracy'], 'colormodels')

# Save the best models for 'Safety'
for clf_name, result in results_safety.items():
    save_best_model(result['Best Model'], clf_name, result['Accuracy'], 'safetymodels')


Saved Random Forest model with accuracy 0.6667 at: colormodels/Random Forest_0.6667.h5
Saved Decision Tree model with accuracy 0.6330 at: colormodels/Decision Tree_0.6330.h5
Saved Support Vector Machine model with accuracy 0.6985 at: colormodels/Support Vector Machine_0.6985.h5
Saved Logistic Regression model with accuracy 0.6985 at: colormodels/Logistic Regression_0.6985.h5
Saved K-Nearest Neighbors model with accuracy 0.6592 at: colormodels/K-Nearest Neighbors_0.6592.h5
Saved Naive Bayes model with accuracy 0.6985 at: colormodels/Naive Bayes_0.6985.h5
Saved AdaBoost model with accuracy 0.7116 at: colormodels/AdaBoost_0.7116.h5
Saved Gradient Boosting model with accuracy 0.6910 at: colormodels/Gradient Boosting_0.6910.h5
Saved Neural Network model with accuracy 0.7022 at: colormodels/Neural Network_0.7022.h5
Saved XGBoost model with accuracy 0.6610 at: colormodels/XGBoost_0.6610.h5
Saved LightGBM model with accuracy 0.6873 at: colormodels/LightGBM_0.6873.h5
Saved Random Forest model w

In [26]:
import os

In [31]:
def load_models(directory):
    models = {}
    for filename in os.listdir(directory):
        if filename.endswith(".h5"):
            model_name, _ = os.path.splitext(filename)
            model_path = os.path.join(directory, filename)
            model = joblib.load(model_path)
            models[model_name] = model
    return models


In [44]:
# Load the saved models for 'Color'
color_models = load_models('colormodels')

# Load the saved models for 'Safety'
safety_models = load_models('safetymodels')

# Function to perform ensemble learning and calculate accuracy
def ensemble_predict(models, X, weights):
    predictions = []
    for model_name, model in models.items():
        try:
            # Try to use predict_proba
            prediction = model.predict_proba(X)[:, 1]
        except AttributeError:
            # If predict_proba is not available, use decision_function or predict
            try:
                prediction = model.decision_function(X)
            except AttributeError:
                prediction = model.predict(X)
        
        predictions.append(prediction * weights[model_name])
    
    ensemble_prediction = sum(predictions) / sum(weights.values())
    return model



In [33]:
# Calculate the weights based on the accuracy from the file name
color_weights = {model_name: float(model_name.split('_')[-1][:-3]) for model_name in color_models}
safety_weights = {model_name: float(model_name.split('_')[-1][:-3]) for model_name in safety_models}

# Assuming X_test is the test set for 'Color' and 'Safety'
# Modify the X_test accordingly for your dataset
X_test_color_ensemble = X_test_scaled_color  # Replace with your test set for 'Color'
X_test_safety_ensemble = X_test_scaled_safety  # Replace with your test set for 'Safety'


In [34]:
# Perform ensemble learning for 'Color'
ensemble_prediction_color = ensemble_predict(color_models, X_test_color_ensemble, color_weights)

# Perform ensemble learning for 'Safety'
ensemble_prediction_safety = ensemble_predict(safety_models, X_test_safety_ensemble, safety_weights)


In [35]:
# Convert the probabilities to binary predictions using a threshold (e.g., 0.5)
threshold = 0.5
ensemble_prediction_color_binary = (ensemble_prediction_color >= threshold).astype(int)
ensemble_prediction_safety_binary = (ensemble_prediction_safety >= threshold).astype(int)


In [36]:
ensemble_accuracy_color = accuracy_score(y_test_color, ensemble_prediction_color_binary)
ensemble_accuracy_safety = accuracy_score(y_test_safety, ensemble_prediction_safety_binary)


In [37]:
print(f"Ensemble Accuracy for 'Color': {ensemble_accuracy_color:.4f}")
print(f"Ensemble Accuracy for 'Safety': {ensemble_accuracy_safety:.4f}")

Ensemble Accuracy for 'Color': 0.6985
Ensemble Accuracy for 'Safety': 1.0000


In [43]:
def save_ensemble_model(ensemble_model, model_name, accuracy, directory):
    file_name = f"{model_name}_ensemble_{accuracy:.4f}.h5"
    file_path = os.path.join(directory, file_name)
    joblib.dump(ensemble_model, file_path)
    print(f"Saved {model_name} ensemble model with accuracy {accuracy:.4f} at: {file_path}")


In [47]:
save_ensemble_model(ensemble_predict(color_models, X_test_color_ensemble, color_weights), 'Color', ensemble_accuracy_color, '')
save_ensemble_model(ensemble_predict(safety_models, X_test_safety_ensemble, safety_weights), 'Safety', ensemble_accuracy_safety, '')


Saved Color ensemble model with accuracy 0.6985 at: Color_ensemble_0.6985.h5
Saved Safety ensemble model with accuracy 1.0000 at: Safety_ensemble_1.0000.h5
